In [ ]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Load the dataset
data = pd.read_csv('ev_fleet_dataset.csv')

# Debugging: Check if 'Year' column exists
if 'Year' not in data.columns:
    print("Error: 'Year' column not found in the dataset.")
    print("Available columns:", data.columns)
    # Create a placeholder 'Year' column for demonstration purposes
    data['Year'] = 2023

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("EV Fleet Dashboard", style={'textAlign': 'center', 'margin-bottom': '20px'}),

    # Filters
    html.Div([
        html.Div([
            html.Label("Select Vehicle ID:"),
            dcc.Dropdown(
                id='vehicle-filter',
                options=[{'label': vehicle, 'value': vehicle} for vehicle in sorted(data['Vehicle_ID'].unique())],
                value=data['Vehicle_ID'].iloc[0],  # Default to the first vehicle
                clearable=False
            )
        ], style={'flex': '1', 'margin-right': '10px'}),

        html.Div([
            html.Label("Select Route Type:"),
            dcc.Dropdown(
                id='route-filter',
                options=[{'label': route, 'value': route} for route in data['Route_Type'].unique()] + [{'label': 'All', 'value': 'All'}],
                value='All',
                clearable=False
            )
        ], style={'flex': '1', 'margin-right': '10px'}),

        html.Div([
            html.Label("Select Year:"),
            dcc.Dropdown(
                id='year-filter',
                options=[{'label': year, 'value': year} for year in sorted(data['Year'].unique())] + [{'label': 'All', 'value': 'All'}],
                value='All',
                clearable=False
            )
        ], style={'flex': '1'})
    ], style={'display': 'flex', 'margin-bottom': '20px'}),

    # KPIs
    html.Div([
        html.Div(id='total-distance', style={
            'background': '#f9f9f9', 'padding': '20px', 'border-radius': '5px', 'textAlign': 'center', 'flex': '1', 'box-shadow': '0 2px 5px rgba(0, 0, 0, 0.1)'}),
        html.Div(id='average-efficiency', style={
            'background': '#f9f9f9', 'padding': '20px', 'border-radius': '5px', 'textAlign': 'center', 'flex': '1', 'box-shadow': '0 2px 5px rgba(0, 0, 0, 0.1)'}),
        html.Div(id='total-cost', style={
            'background': '#f9f9f9', 'padding': '20px', 'border-radius': '5px', 'textAlign': 'center', 'flex': '1', 'box-shadow': '0 2px 5px rgba(0, 0, 0, 0.1)'}),
    ], style={'display': 'flex', 'gap': '20px', 'margin-bottom': '20px'}),

    # Separate Graphs for Energy Consumption and Operational Costs
    html.Div([
        html.H3("Energy Consumption Over Time", style={'textAlign': 'center', 'margin-bottom': '10px'}),
        dcc.Graph(id='energy-consumption-graph')
    ], style={'margin-bottom': '20px'}),

    html.Div([
        html.H3("Operational Cost Over Time", style={'textAlign': 'center', 'margin-bottom': '10px'}),
        dcc.Graph(id='operational-cost-graph')
    ])
])

# Callbacks to update KPIs and graphs
@app.callback([
    Output('total-distance', 'children'),
    Output('average-efficiency', 'children'),
    Output('total-cost', 'children'),
    Output('energy-consumption-graph', 'figure'),
    Output('operational-cost-graph', 'figure')
], [
    Input('vehicle-filter', 'value'),
    Input('route-filter', 'value'),
    Input('year-filter', 'value')
])
def update_dashboard(selected_vehicle, selected_route, selected_year):
    # Filter data based on selected vehicle, route type, and year
    filtered_data = data[data['Vehicle_ID'] == selected_vehicle]
    if selected_route != 'All':
        filtered_data = filtered_data[filtered_data['Route_Type'] == selected_route]
    if selected_year != 'All':
        filtered_data = filtered_data[filtered_data['Year'] == selected_year]

    # Compute KPIs
    total_distance = filtered_data['Distance_Travelled_km'].sum()
    average_efficiency = filtered_data['Efficiency_kWh_per_km'].mean()
    total_cost = filtered_data['Operational_Cost_USD'].sum()

    # Create graphs
    energy_consumption_fig = px.line(
        filtered_data, x='Year', y='Energy_Consumption_kWh',
        title=f'Energy Consumption for Vehicle {selected_vehicle}',
        labels={'Energy_Consumption_kWh': 'Energy Consumption (kWh)', 'Year': 'Year'},
        markers=True
    )
    energy_consumption_fig.update_layout(
        title_font_size=18,
        xaxis_title='Year',
        yaxis_title='Energy Consumption (kWh)',
        plot_bgcolor='#f9f9f9',
        paper_bgcolor='#ffffff',
        margin=dict(l=40, r=40, t=40, b=40)
    )

    operational_cost_fig = px.line(
        filtered_data, x='Year', y='Operational_Cost_USD',
        title=f'Operational Cost for Vehicle {selected_vehicle}',
        labels={'Operational_Cost_USD': 'Operational Cost (USD)', 'Year': 'Year'},
        markers=True
    )
    operational_cost_fig.update_layout(
        title_font_size=18,
        xaxis_title='Year',
        yaxis_title='Operational Cost (USD)',
        plot_bgcolor='#f9f9f9',
        paper_bgcolor='#ffffff',
        margin=dict(l=40, r=40, t=40, b=40)
    )

    # Format KPI display
    kpi_distance = f"Total Distance: {total_distance:.2f} km"
    kpi_efficiency = f"Average Efficiency: {average_efficiency:.2f} kWh/km"
    kpi_cost = f"Total Cost: ${total_cost:.2f}"

    return kpi_distance, kpi_efficiency, kpi_cost, energy_consumption_fig, operational_cost_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)